# Finetuning the model

In [2]:
!pip install -q -U pyarrow==14.0.1
!pip install -q -U fsspec==2023.10.0

!pip install -q -U bitsandbytes==0.42.0
!pip install -q -U peft==0.8.2
!pip install -q -U trl==0.7.10
!pip install -q -U accelerate==0.27.1
!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.38.0
!pip install python-dotenv

print("done")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 12.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2023.10.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 22.0 MB/s eta 0:00:00
   ━━━━━

In [1]:
from huggingface_hub import login

login()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
import accelerate
import bitsandbytes as bnb
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Set quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16)

# Load the model and tokenizer
model_id = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [10]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("soufyane/DATA_SCIENCE_QA")
dataset


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Question', 'Answer'],
        num_rows: 1070
    })
})

In [4]:
dataset["train"][0]

{'Unnamed: 0': 0,
 'Question': 'What is under-fitting and overfitting in machine learning?',
 'Answer': "Underfitting is when a model is too simple, and overfitting is when it's too complex, making it perform poorly on new data."}

In [11]:
# Convert HuggingFace dataset to dataframe
df = dataset["train"].to_pandas()
df.head()

,Unnamed: 0,Question,Answer
0,0,What is under-fitting and overfitting in machi...,"Underfitting is when a model is too simple, an..."
1,1,Can you explain what a false positive and a fa...,A false positive incorrectly indicates a condi...
2,2,Clarify the concept of Phase IV.,"Phase IV studies, also known as post-marketing..."
3,3,What is semi-supervised learning described in ...,Semi-supervised learning integrates both label...
4,4,Discuss the parallelization of training in gra...,Parallelizing training of a gradient boosting ...


In [12]:
# Define function to generate the prompt

def generate_prompt(data_point):
  # Generate prompt
    prefix_text = 'Answer the following question. Write the answer in between <answer></answer>.'

    if data_point["Question"]:
      prompt = f"""<start_of_turn>user {prefix_text} {data_point["Question"]} <end_of_turn>\n<start_of_turn>model {data_point["Answer"]} <end_of_turn>"""
      return prompt

# Add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset["train"]]
dataset = dataset["train"].add_column("prompt", text_column)
dataset

Dataset({
    features: ['Unnamed: 0', 'Question', 'Answer', 'prompt'],
    num_rows: 1070
})

In [ ]:
print(dataset[0]['prompt'])

<start_of_turn>user Answer the following question. Write the answer in between <answer></answer>. What types of artificial intelligence exist? <end_of_turn>
<start_of_turn>model AI types vary by capability: Weak AI performs specific tasks, General AI matches human intellect, and Artificial Superhuman Intelligence surpasses human intelligence. <end_of_turn>


In [13]:
# Shuffle dataset
dataset = dataset.shuffle(seed=1234)
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/1070 [00:00<?, ? examples/s]

In [15]:
# Train-Test Split
dataset = dataset.train_test_split(test_size=0.1)
train_data = dataset["train"]
test_data = dataset["test"]

print(train_data)
print(test_data)

Dataset({
    features: ['Unnamed: 0', 'Question', 'Answer', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 963
})
Dataset({
    features: ['Unnamed: 0', 'Question', 'Answer', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 107
})


Low-Rank Adaptation for Large Language Models (LoRA)

In [ ]:
# Automated selection of target modules
import bitsandbytes as bnb

def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)
print(modules)

['gate_proj', 'v_proj', 'q_proj', 'o_proj', 'down_proj', 'up_proj', 'k_proj']


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, get_peft_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
     

In [ ]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
# Number of trainable parameters
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | Total: {total} | Percentage: {100 * trainable / total:.4f}%")

Trainable: 78446592 | Total: 2584619008 | Percentage: 3.0351%


Train the Model

In [ ]:
import transformers
from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=lora_config,
    dataset_text_field="prompt",
    max_seq_length=2500,
    args = transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=100,
        num_train_epochs=2,
        learning_rate=2e-4,
        logging_steps=20,
        output_dir='outputs',
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        report_to="none"
    ),
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/963 [00:00<?, ? examples/s]

Map:   0%|          | 0/107 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.a

Step,Training Loss
20,3.562700
40,2.017100
60,1.863000
80,1.790300
100,1.719400


TrainOutput(global_step=100, training_loss=2.190466995239258, metrics={'train_runtime': 327.4682, 'train_samples_per_second': 1.221, 'train_steps_per_second': 0.305, 'total_flos': 346061437427712.0, 'train_loss': 2.190466995239258, 'epoch': 0.42})

In [ ]:
from peft import PeftModel

new_model = "gemma-2b-it-ft-data-science-qa"
trainer.model.save_pretrained(new_model)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model = PeftModel.from_pretrained(base_model, new_model)
merged_model = merged_model.merge_and_unload()

# Push the model and tokenizer to the HuggingFace Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/cherrymaecaracas/gemma-2b-it-ft-data-science-qa/commit/c922cc3e5e34f78ba92350f90a4678c75a15dd59', commit_message='Upload tokenizer', commit_description='', oid='c922cc3e5e34f78ba92350f90a4678c75a15dd59', pr_url=None, pr_revision=None, pr_num=None)

# Evaluation Metrics

In [19]:
# Load/define base (non-finetuned) vs finetuned models for comparison

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from math import exp
import bitsandbytes as bnb  # Ensure bitsandbytes is installed if using quantization
from peft import PeftModel  # If using PEFT models
from huggingface_hub import login

# Define your Hugging Face username and model IDs
username = "cherrymaecaracas"
finetuned_model_id = f"{username}/gemma-2b-it-ft-data-science-qa"
base_model_id = "google/gemma-2b-it"

# Set the qunatization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map={"":0})
base_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_eos_token=True)

# Load the Finetuned Model and Tokenizer from HF
finetuned_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_id)
finetuned_model = AutoModelForCausalLM.from_pretrained(
                  finetuned_model_id,
                  torch_dtype=torch.float16,
                  device_map="auto"
)

# Ensure the model is in evaluation mode
base_model.eval()
finetuned_model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRM

Semantic Similarity

In [16]:
test_data

Dataset({
    features: ['Unnamed: 0', 'Question', 'Answer', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 107
})

In [17]:
test_data_subset = test_data.shuffle(seed=42).select(range(10))  # Get 10 random rows

In [21]:
import torch
from torch.nn.functional import cosine_similarity

def calculate_semantic_similarity(model, tokenizer, dataset):
  similarities = []
  model.eval

  with torch.no_grad():
    for example in dataset:
      # Encode the input
      input_ids = tokenizer.encode(example['prompt'], return_tensors='pt').to('cuda')
      output = model

      # Generate output from the model
      output_ids = model.generate(input_ids, max_new_tokens=150)
      generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

      # Tokenize generated and reference texts
      gen_inputs = tokenizer(generated_text, return_tensors='pt').to("cuda")
      ref_inputs = tokenizer(example["Answer"], return_tensors='pt').to("cuda")

      # Get embeddings from the last hidden state
      gen_outputs = model(**gen_inputs, output_hidden_states=True, return_dict=True)
      ref_outputs = model(**ref_inputs, output_hidden_states=True, return_dict=True)

      # Average pooling of the embeddings
      gen_embedding = gen_outputs.hidden_states[-1].mean(dim=1).squeeze()
      ref_embedding = ref_outputs.hidden_states[-1].mean(dim=1).squeeze()

      # Calculate cosine similarity
      cosine_score = cosine_similarity(gen_embedding, ref_embedding, dim=0)
      similarities.append(cosine_score.item())

  average_similarity = sum(similarities) / len(similarities)
  return average_similarity

In [22]:
base_semantic_similarity = calculate_semantic_similarity(base_model, base_tokenizer, test_data_subset)
finetuned_semantic_similarity = calculate_semantic_similarity(finetuned_model, finetuned_tokenizer, test_data_subset)

print(f"Non-finetuned Model Semantic Similarity: {base_semantic_similarity}")
print(f"Finetuned Model Semantic Similarity: {finetuned_semantic_similarity}")

Non-finetuned Model Semantic Similarity: 0.94931640625
Finetuned Model Semantic Similarity: 0.978759765625
